# Challenge5 不動産サイトからスクレイピング

### 【ミッション】  
● RequestでHTML取得  
● HTMLデータから欲しい情報を取得  
● 取得データをPandasで構造化して保存  

### 【コード完成形】

In [8]:
# ライブラリインポート
from time import sleep
import requests
from bs4 import BeautifulSoup
import pandas as pd
import gspread 
from google.oauth2 import service_account
import json

# スプレッドシートの認証設定
SP_CREDENTIAL_FILE = 'gspread-test-399313-ca3a69c4d75e.json' #個人の認証キーのjsonファイルのパスを記入
SP_COPE = [
    'https://www.googleapis.com/auth/drive',
    'https://spreadsheets.google.com/feeds'
]
credentials = service_account.Credentials.from_service_account_file(SP_CREDENTIAL_FILE, scopes=SP_COPE)
gc = gspread.authorize(credentials)

#スプレッドシート/シート指定
SP_SHEET_KEY = '1474-fT6RkuGOvPwjyCJIEtD6qrNjI6EP2bw3vtHvLBc' #川名のスプレッドシート
SP_SHEET = 'demo'

# 情報格納のための空配列を用意
p_name = []
p_locate = []
p_station = []
p_rent = []
p_security = []
p_area = []
p_feature = []
p_floor = []

# HTML取得
URL = 'https://www.temponw.com/area_search/result?prefectures_code=13&name_j=%E6%9D%B1%E4%BA%AC%E9%83%BD&total_groups=2&cities%5B%5D=104&page={}'

# 複数ページ取得のためのループ処理
for i in range(1,11):

    # 変数trget_urlに、アクセス先のURLを格納する
    target_url = URL.format(i)

    #サーバー負荷低減のための1s待機
    sleep(1)

    #print()してtrget_url確認
    print(target_url)

    # リクエスト
    res = requests.get(target_url)

    # リクエスト結果を確認
    #result = res.text
    #print(result)

    # 文字コード変換 (→ utf-8 )
    res.encoding = 'utf-8'

    # 文字化けしていないかhtmlを確認
    #print(res.text)

    # BeautifulSoup(解析したいデータ,解析する方法)を指定し、soupに代入
    soup = BeautifulSoup(res.text, "html.parser")
    #print(soup)

    # sectionデータ(1物件分データ)の取得
    property_section = soup.select('div.result-contents.sm-hidden')

    # 取得したセクションすべての数を確認します。
    #print(len(property_section))

    # property_sectionからsectionを1つずつ取り出してpsに代入。
    for ps in property_section:
        ps1 = ps.select('label.checkbox-container')[0].text.replace("\n","")
        ps2 = ps.select('div.result-detail-row')[1].text.replace("\n所在地\n","").replace("\n","")
        ps3 = ps.select('div.result-detail-row')[2].text.replace("\n最寄駅\n","").replace("\n","").replace(" ","").replace("\u3000"," ").replace("、他...","")
        ps4 = ps.select('div.result-detail-row.border-top-line')[0].text.replace("\n賃料\n","").replace("月額 ","").replace("万円","").replace(" 税込","").replace("\n","")
        ps5 = ps.select('div.result-detail-row')[4].text.replace("\n保証金/礼金\n\n","").replace("\n","").replace("保","").replace("ヶ月","").replace("-","0")
        ps6 = ps.select('div.result-detail-row')[5].text.replace("\n面積\n","").replace("\n","").replace(" ","").split("m")[0]
        ps7 = ps.select('div.result-detail-row')[6].text.replace("使用階","").replace("/部屋番号","").replace("\n","").replace(" ","")
        Features = []
        div_elements = ps.select('div.detail-grey')
        for div_element in div_elements:
            try:
                Features.append(div_element.text)
            except Exception as e:
                Features.append("Error")
        ps8 = ", ".join(Features)

        # それぞれ抽出したデータを配列に追加
        p_name.append(ps1)
        p_locate.append(ps2)
        p_station.append(ps3)
        p_rent.append(ps4)
        p_security.append(ps5)
        p_area.append(ps6)
        p_floor.append(ps7)
        p_feature.append(ps8)
        
    # 物件数確認
    print('検索物件数', len(p_name))

# データ定義
data_list = {
    "物件名" : p_name,
    "使用階/部屋番号" : p_floor,
    "床面積(m2)" : p_area,
    "賃料(万円)" : p_rent,
    "保証金/礼金(月)" : p_security,
    "所在地" : p_locate,
    "最寄駅" : p_station,
    "用途・特徴" : p_feature,
}

# データフレーム作成
df = pd.DataFrame(data_list)

# 重複データ処理
df.drop_duplicates(inplace=True)
df.reset_index(drop=True,inplace=True)
# display(df)

# サロン向け物件のみ抽出
filtered_df = df[df['用途・特徴'].str.contains('サロン')]
#display(filtered_df.head) 

# 不要化した「用途・特徴」列を削除
filtered_df = filtered_df.drop(columns=['用途・特徴'])
#filtered_df

# 保証金、礼金を分割する
filtered_df[['保証金(月)', '礼金(月)']] = filtered_df['保証金/礼金(月)'].str.split('礼', n=1, expand=True)
filtered_df.drop(columns=['保証金/礼金(月)'], inplace=True)
#filtered_df

# csv保存
#filtered_df.to_csv('東京都新宿区の貸店舗・テナント物件一覧_店舗ネットワーク.csv', index=False)

# スプレッドシート/シートを選択して初期化既存データ削除）
sh = gc.open_by_key(SP_SHEET_KEY)
worksheet = sh.worksheet(SP_SHEET)
worksheet.clear() 

# DataFrameをシートに書き込む
worksheet.update([filtered_df.columns.values.tolist()] + filtered_df.values.tolist())

print('スプレッドシートへの書き込みが完了しました')

https://www.temponw.com/area_search/result?prefectures_code=13&name_j=%E6%9D%B1%E4%BA%AC%E9%83%BD&total_groups=2&cities%5B%5D=104&page=1
検索物件数 10
https://www.temponw.com/area_search/result?prefectures_code=13&name_j=%E6%9D%B1%E4%BA%AC%E9%83%BD&total_groups=2&cities%5B%5D=104&page=2
検索物件数 20
https://www.temponw.com/area_search/result?prefectures_code=13&name_j=%E6%9D%B1%E4%BA%AC%E9%83%BD&total_groups=2&cities%5B%5D=104&page=3
検索物件数 30
https://www.temponw.com/area_search/result?prefectures_code=13&name_j=%E6%9D%B1%E4%BA%AC%E9%83%BD&total_groups=2&cities%5B%5D=104&page=4
検索物件数 40
https://www.temponw.com/area_search/result?prefectures_code=13&name_j=%E6%9D%B1%E4%BA%AC%E9%83%BD&total_groups=2&cities%5B%5D=104&page=5
検索物件数 50
https://www.temponw.com/area_search/result?prefectures_code=13&name_j=%E6%9D%B1%E4%BA%AC%E9%83%BD&total_groups=2&cities%5B%5D=104&page=6
検索物件数 60
https://www.temponw.com/area_search/result?prefectures_code=13&name_j=%E6%9D%B1%E4%BA%AC%E9%83%BD&total_groups=2&cities%5B%5

c:\Users\noriyasukawana\AppData\Local\Programs\Python\Python311\Lib\site-packages\gspread\worksheet.py:1069: UserWarning: [Deprecated][in version 6.0.0]: method signature will change to: 'Worksheet.update(value = [[]], range_name=)' arguments 'range_name' and 'values' will swap, values will be mandatory of type: 'list(list(...))'
  warnings.warn(


スプレッドシートへの書き込みが完了しました


In [9]:
# データ確認
filtered_df

,物件名,使用階/部屋番号,床面積(m2),賃料(万円),所在地,最寄駅,保証金(月),礼金(月)
0,西早稲田１階店舗,1,80.44,41.8,東京都新宿区西早稲田1-23-13,都電荒川線 早稲田,6.00,-
1,ＫＭプラザ ３Ｆ,3/302-303,57.82,18.7,東京都新宿区大久保１丁目16-27,ＪＲ山手線 新大久保,2.00,-
3,小玉ビル,4/403,20.91,11,東京都新宿区西新宿7丁目19-11,東京メトロ丸ノ内線 西新宿,-,1.10
6,ＭＡフラットビル,1,48.00,26.18,東京都新宿区高田馬場2-7-13,ＪＲ山手線 高田馬場,10.00,2.20
7,第二サンパークビル １階,1,59.10,181.5,東京都新宿区新宿3-21-4,ＪＲ山手線 新宿,10.00,1.00
8,四谷林ビル,1棟/1階~3階,256.15,132,東京都新宿区四谷一丁目18-11,東京メトロ丸ノ内線 四ツ谷,6.00,-
9,恩田セントラルビル ４階,4/401,62.35,62.24,東京都新宿区新宿3-3-3,東京メトロ副都心線 新宿三丁目,10.00,1.10
10,ｒａｕｍ神楽坂,1,6.21,4.62,東京都新宿区東五軒町4番14号,東京メトロ東西線 神楽坂,-,1.10
12,三宅ビル Ｂ１階,B1,110.75,63.74,東京都新宿区四谷1丁目9番地,東京メトロ丸ノ内線 四谷三丁目,10.00,1.10
13,三宅ビル ２階,2,99.34,66.11,東京都新宿区四谷1丁目9番地,東京メトロ丸ノ内線 四谷三丁目,10.00,1.10


### 【RequestでHTML取得】

>0. 検索する情報の仮決め

不動産サイト：店舗ネットワークから、東京都新宿区の貸店舗・テナント物件一覧  
https://www.temponw.com/area_search/result?prefectures_code=13&name_j=%E6%9D%B1%E4%BA%AC%E9%83%BD&total_groups=2&cities%5B%5D=104

>1. 規約の確認


✔ https://www.temponw.com/robots.txt  
「 Disallow:/rent_store 」を検索 → 無し  
✔ プライバシーポリシー  
スクレイピング・クローリング禁止の記載なし


>2. 環境構築

ターミナルにて、  
 ● pip install requests  
 ● pip install beautifulsoup4  
 ● pip install gspread  
 ● pip install google-auth


>3. ライブラリインポート

In [56]:
import requests
from bs4 import BeautifulSoup
from time import sleep

>4. htmlデータ取得

In [57]:
REQUESTS_URL = 'https://www.temponw.com/area_search/result?prefectures_code=13&name_j=%E6%9D%B1%E4%BA%AC%E9%83%BD&total_groups=2&cities%5B%5D=104&page=1'
res = requests.get(REQUESTS_URL)

>5. 文字化けチェック

In [58]:
result = res.text
print(result)

<!DOCTYPE html>
<html lang="ja">

<head>
    <!-- Google tag (gtag.js) -->
    <script async src="https://www.googletagmanager.com/gtag/js?id=G-4YFTKLRRJ2"></script>
    <script>
    window.dataLayer = window.dataLayer || [];
    function gtag(){dataLayer.push(arguments);}
    gtag('js', new Date());
    gtag('config', 'G-4YFTKLRRJ2');
    gtag('config', 'UA-209055526-2');
    </script>
    <!-- Google Tag Manager -->
    <script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
    new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
    j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
    'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
    })(window,document,'script','dataLayer','GTM-KDJFPCW');</script>
    <!-- End Google Tag Manager -->
    <meta charset="UTF-8"/>
    <meta name="csrf-token" content="u5BgKLPyJItmX1flB5rvDQXidV4w5R11yt0QPhac">
    <meta name="viewport" content="width=device-width, 

>6. 文字コードの確認・変換

In [59]:
# 文字コードの確認
print(res.encoding)

UTF-8


In [60]:
# 文字コードの変換 (→ utf-8 )
res.encoding = 'utf-8'
# 文字化けしていないかhtmlを確認
print(res.text)

<!DOCTYPE html>
<html lang="ja">

<head>
    <!-- Google tag (gtag.js) -->
    <script async src="https://www.googletagmanager.com/gtag/js?id=G-4YFTKLRRJ2"></script>
    <script>
    window.dataLayer = window.dataLayer || [];
    function gtag(){dataLayer.push(arguments);}
    gtag('js', new Date());
    gtag('config', 'G-4YFTKLRRJ2');
    gtag('config', 'UA-209055526-2');
    </script>
    <!-- Google Tag Manager -->
    <script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
    new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
    j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
    'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
    })(window,document,'script','dataLayer','GTM-KDJFPCW');</script>
    <!-- End Google Tag Manager -->
    <meta charset="UTF-8"/>
    <meta name="csrf-token" content="u5BgKLPyJItmX1flB5rvDQXidV4w5R11yt0QPhac">
    <meta name="viewport" content="width=device-width, 

### 【HTMLデータから欲しい情報を習得】

>7. BeautifulSoupでhtml解析

In [66]:
# BeautifulSoup(解析したいデータ,解析する方法)を指定し、データをsoupに代入します。
soup = BeautifulSoup(res.text, "html.parser")
soup

<!DOCTYPE html>

<html lang="ja">
<head>
<!-- Google tag (gtag.js) -->
<script async="" src="https://www.googletagmanager.com/gtag/js?id=G-4YFTKLRRJ2"></script>
<script>
    window.dataLayer = window.dataLayer || [];
    function gtag(){dataLayer.push(arguments);}
    gtag('js', new Date());
    gtag('config', 'G-4YFTKLRRJ2');
    gtag('config', 'UA-209055526-2');
    </script>
<!-- Google Tag Manager -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
    new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
    j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
    'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
    })(window,document,'script','dataLayer','GTM-KDJFPCW');</script>
<!-- End Google Tag Manager -->
<meta charset="utf-8"/>
<meta content="GuUa1ucZR8eFd7nsZAISEnuBYXvbG0ARY159c3x9" name="csrf-token"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href=

>8. 欲しい情報を1つずつしか持たないhtmlを抽出する

● HP上で右クリック  
● 検証　を選択 → HTMLコードが見れるようになる  
● 「ページ上の検査対象となる要素の選択」ボタンをクリック  


●　物件名：label.checkbox-container    
●　所在地：div.result-detail-row  
●　最寄駅：div.result-detail-row  
●　賃料：div.result-detail-row.border-top-line  
●　管理費：div.result-detail-row  
●　面積：div.result-detail-row  
●　使用階：div.result-detail-row  
●　用途：div.detail-grey  

![Alt text](image-1.png)

In [62]:
# すべてのsectionタグを取得します。
property_section = soup.select('div.result-contents.sm-hidden')
# 取得したセクションすべての数を確認します。
len(property_section)

10

>9. 1つの物件の欲しい情報を取得

In [ ]:
ps = property_section[0]
# 物件名
print(ps.select('label.checkbox-container')[0].text)
# 所在地
print(ps.select('div.result-detail-row')[1].text)
# 最寄駅
print(ps.select('div.result-detail-row')[2].text)
# 賃料
print(ps.select('div.result-detail-row.border-top-line')[0].text)
# 面積
print(ps.select('div.result-detail-row')[5].text)
# 店舗特徴
print(ps.select('div.detail-grey'))

>10. データ整理

In [ ]:
# 所在地(記載整理)
ps.select('div.result-detail-row')[1].text.replace("\n所在地\n","").replace("\n","")

In [ ]:
# 最寄駅(記載整理)
ps.select('div.result-detail-row')[2].text.replace("\n最寄駅\n","").replace("\n","").replace(" ","").replace("\u3000"," ").replace("、他...","")

In [ ]:
# 賃料(万円)
ps.select('div.result-detail-row.border-top-line')[0].text.replace("\n賃料\n","").replace("月額 ","").replace("万円","").replace(" 税込\n","")

In [ ]:
# 面積(記載整理)
ps.select('div.result-detail-row')[5].text.replace("\n面積\n","").replace("\n","").replace(" ","").split("m")[0]

In [ ]:
# 美容室利用可否
Features = []
div_elements = ps.select('div.detail-grey')
for div_element in div_elements:
    try:
        Features.append(div_element.text)
    except Exception as e:
        Features.append("Error")
if "理美容・サロン" in Features:
    result = 1
else:
    result = 0
result

In [ ]:
#まとめ

ps = property_section[0]

#物件名
print(ps.select('label.checkbox-container')[0].text.replace("\n",""))
#所在地
print(ps.select('div.result-detail-row')[1].text.replace("\n所在地\n","").replace("\n",""))
#最寄駅
print(ps.select('div.result-detail-row')[2].text.replace("\n最寄駅\n","").replace("\n","").replace(" ","").replace("\u3000"," ").replace("、他...",""))
#賃料(万円)
print(ps.select('div.result-detail-row.border-top-line')[0].text.replace("\n賃料\n","").replace("月額 ","").replace("万円","").replace(" 税込\n",""))
#面積
print(ps.select('div.result-detail-row')[5].text.replace("\n面積\n","").replace("\n","").replace(" ","").split("m")[0])
#美容室利用可否
Features = []
div_elements = ps.select('div.detail-grey')
for div_element in div_elements:
    try:
        Features.append(div_element.text)
    except Exception as e:
        Features.append("Error")
Feature = ", ".join(Features)
Feature


In [ ]:
# 情報格納のための空配列を用意
p_name = []
p_locate = []
p_station = []
p_rent = []
p_area = []
p_feature = []

# property_sectionから任意のsectionを1つ選んでpsに代入。
ps = property_section[5]

# sectionから欲しい情報を抽出
ps1 = ps.select('label.checkbox-container')[0].text.replace("\n","")
ps2 = ps.select('div.result-detail-row')[1].text.replace("\n所在地\n","").replace("\n","")
ps3 = ps.select('div.result-detail-row')[2].text.replace("\n最寄駅\n","").replace("\n","").replace(" ","").replace("\u3000"," ").replace("、他...","")
ps4 = ps.select('div.result-detail-row.border-top-line')[0].text.replace("\n賃料\n","").replace("月額 ","").replace("万円","").replace(" 税込\n","")
ps5 = ps.select('div.result-detail-row')[5].text.replace("\n面積\n","").replace("\n","").replace(" ","").split("m")[0]
Features = []
div_elements = ps.select('div.detail-grey')
for div_element in div_elements:
    try:
        Features.append(div_element.text)
    except Exception as e:
        Features.append("Error")
ps6 = ", ".join(Features)

# それぞれ抽出したデータを配列に追加
p_name.append(ps1)
p_locate.append(ps2)
p_station.append(ps3)
p_rent.append(ps4)
p_area.append(ps5)
p_feature.append(ps6)


In [ ]:
#配列の中身確認
print(p_name)
print(p_locate)
print(p_station)
print(p_rent)
print(p_area)
print(p_feature)

>11. 物件データをまとめて取得

In [ ]:
# 情報格納のための空配列を用意
p_name = []
p_locate = []
p_station = []
p_rent = []
p_area = []
p_feature = []

# property_sectionからsectionを1つずつ取り出してpsに代入。
for ps in property_section:
    ps1 = ps.select('label.checkbox-container')[0].text.replace("\n","")
    ps2 = ps.select('div.result-detail-row')[1].text.replace("\n所在地\n","").replace("\n","")
    ps3 = ps.select('div.result-detail-row')[2].text.replace("\n最寄駅\n","").replace("\n","").replace(" ","").replace("\u3000"," ").replace("、他...","")
    ps4 = ps.select('div.result-detail-row.border-top-line')[0].text.replace("\n賃料\n","").replace("月額 ","").replace("万円","").replace(" 税込\n","")
    ps5 = ps.select('div.result-detail-row')[5].text.replace("\n面積\n","").replace("\n","").replace(" ","").split("m")[0]
    Features = []
    div_elements = ps.select('div.detail-grey')
    for div_element in div_elements:
        try:
            Features.append(div_element.text)
        except Exception as e:
            Features.append("Error")
    ps6 = ", ".join(Features)

    # それぞれ抽出したデータを配列に追加
    p_name.append(ps1)
    p_locate.append(ps2)
    p_station.append(ps3)
    p_rent.append(ps4)
    p_area.append(ps5)
    p_feature.append(ps6)

### 【取得データをPandasで構造化して保存】

>12. 抽出した配列をDataFrame化

In [ ]:
# データ定義
data_list = {
    "物件名" : p_name,
    "所在地" : p_locate,
    "最寄駅" : p_station,
    "賃料(万円)" : p_rent,
    "床面積(m2)" : p_area,
    "用途・特徴" : p_feature,
}

In [ ]:
import pandas as pd
df = pd.DataFrame(data_list)
df

>13. 重複チェック

In [ ]:
df.duplicated()

>14. 重複データ削除

In [ ]:
# drop_duplicates()で重複したデータを削除してくれます。
# drop_duplicates(inplace=True)とすることで、処理したデータを出力だけでなく、出力したデータを元のdfに代入して変更してくれます。
df.drop_duplicates(inplace=True)
df

In [ ]:
# 列の認識番号であるindexが列の順番と一致していないので、reset_indexで番号を振り直します。
# reset_index(drop=True,inplace=True)とすることで、元のindexを削除して新たに生成したindexを作成し、元のデータを更新してくれます。
df.reset_index(drop=True,inplace=True)
df

In [ ]:
# csv保存
df.to_csv('観光地情報.csv', index=False)

>15. 全コード集約

In [71]:
# ライブラリインポート
from time import sleep
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 情報格納のための空配列を用意
p_name = []
p_locate = []
p_station = []
p_rent = []
p_area = []
p_feature = []

# HTML取得
URL = 'https://www.temponw.com/area_search/result?prefectures_code=13&name_j=%E6%9D%B1%E4%BA%AC%E9%83%BD&total_groups=2&cities%5B%5D=104&page={}'

# 複数ページ取得のためのループ処理
for i in range(1,11):

    # 変数trget_urlに、アクセス先のURLを格納する
    target_url = URL.format(i)

    #サーバー負荷低減のための1s待機
    sleep(1)

    #print()してtrget_url確認
    print(target_url)

    # リクエスト
    res = requests.get(target_url)

    # リクエスト結果を確認
    #result = res.text
    #print(result)

    # 文字コード変換 (→ utf-8 )
    res.encoding = 'utf-8'

    # 文字化けしていないかhtmlを確認
    #print(res.text)

    # BeautifulSoup(解析したいデータ,解析する方法)を指定し、soupに代入
    soup = BeautifulSoup(res.text, "html.parser")
    #print(soup)

    # sectionデータ(1物件分データ)の取得
    property_section = soup.select('div.result-contents.sm-hidden')

    # 取得したセクションすべての数を確認します。
    #print(len(property_section))

    # property_sectionからsectionを1つずつ取り出してpsに代入。
    for ps in property_section:
        ps1 = ps.select('label.checkbox-container')[0].text.replace("\n","")
        ps2 = ps.select('div.result-detail-row')[1].text.replace("\n所在地\n","").replace("\n","")
        ps3 = ps.select('div.result-detail-row')[2].text.replace("\n最寄駅\n","").replace("\n","").replace(" ","").replace("\u3000"," ").replace("、他...","")
        ps4 = ps.select('div.result-detail-row.border-top-line')[0].text.replace("\n賃料\n","").replace("月額 ","").replace("万円","").replace(" 税込","").replace("\n","")
        ps5 = ps.select('div.result-detail-row')[5].text.replace("\n面積\n","").replace("\n","").replace(" ","").split("m")[0]
        Features = []
        div_elements = ps.select('div.detail-grey')
        for div_element in div_elements:
            try:
                Features.append(div_element.text)
            except Exception as e:
                Features.append("Error")
        ps6 = ", ".join(Features)

        # それぞれ抽出したデータを配列に追加
        p_name.append(ps1)
        p_locate.append(ps2)
        p_station.append(ps3)
        p_rent.append(ps4)
        p_area.append(ps5)
        p_feature.append(ps6)
        
    # 物件数確認
    print('検索物件数', len(p_name))

# データ定義
data_list = {
    "物件名" : p_name,
    "所在地" : p_locate,
    "最寄駅" : p_station,
    "賃料(万円)" : p_rent,
    "床面積(m2)" : p_area,
    "用途・特徴" : p_feature,
}

# データフレーム作成
df = pd.DataFrame(data_list)

# 重複データ処理
df.drop_duplicates(inplace=True)
df.reset_index(drop=True,inplace=True)
display(df)

# csv保存
df.to_csv('東京都新宿区の貸店舗・テナント物件一覧_店舗ネットワーク.csv', encoding='utf-8', index=False)



https://www.temponw.com/area_search/result?prefectures_code=13&name_j=%E6%9D%B1%E4%BA%AC%E9%83%BD&total_groups=2&cities%5B%5D=104&page=1
検索物件数 10
https://www.temponw.com/area_search/result?prefectures_code=13&name_j=%E6%9D%B1%E4%BA%AC%E9%83%BD&total_groups=2&cities%5B%5D=104&page=2
検索物件数 20
https://www.temponw.com/area_search/result?prefectures_code=13&name_j=%E6%9D%B1%E4%BA%AC%E9%83%BD&total_groups=2&cities%5B%5D=104&page=3
検索物件数 30
https://www.temponw.com/area_search/result?prefectures_code=13&name_j=%E6%9D%B1%E4%BA%AC%E9%83%BD&total_groups=2&cities%5B%5D=104&page=4
検索物件数 40
https://www.temponw.com/area_search/result?prefectures_code=13&name_j=%E6%9D%B1%E4%BA%AC%E9%83%BD&total_groups=2&cities%5B%5D=104&page=5
検索物件数 50
https://www.temponw.com/area_search/result?prefectures_code=13&name_j=%E6%9D%B1%E4%BA%AC%E9%83%BD&total_groups=2&cities%5B%5D=104&page=6
検索物件数 60
https://www.temponw.com/area_search/result?prefectures_code=13&name_j=%E6%9D%B1%E4%BA%AC%E9%83%BD&total_groups=2&cities%5B%5

,物件名,所在地,最寄駅,賃料(万円),床面積(m2),用途・特徴
0,西早稲田１階店舗,東京都新宿区西早稲田1-23-13,都電荒川線 早稲田,41.8,80.44,"軽飲食, 物販, 理美容・サロン, クリニック・介護, アミューズメント, 事務所, その他..."
1,ＫＭプラザ ３Ｆ,東京都新宿区大久保１丁目16-27,ＪＲ山手線 新大久保,18.7,57.82,"理美容・サロン, クリニック・介護, アミューズメント, 塾・スクール, 事務所, 24時間..."
2,ヒルトンホテル,東京都新宿区西新宿6-6-2,東京メトロ丸ノ内線 西新宿,31.51,20.84,"物販, 24時間利用可"
3,小玉ビル,東京都新宿区西新宿7丁目19-11,東京メトロ丸ノ内線 西新宿,11,20.91,"理美容・サロン, 塾・スクール, 事務所, 角地"
4,Ｎ．Ｋビル,東京都新宿区西早稲田2-16-17,東京メトロ副都心線 西早稲田,17.6,49.59,"軽飲食, 物販, 塾・スクール, 事務所, 角地"
...,...,...,...,...,...,...
85,灯ビル,東京都新宿区歌舞伎町１丁目26,西武鉄道新宿線 西武新宿,147.23,201.13,"重飲食, ラーメン・中華料理, カレー・多国籍料理, 和食・寿司, うどん・そば, 焼き鳥,..."
86,（仮称）新宿区富久町計画,東京都新宿区富久町1,都営新宿線 曙橋,69.1812,115.52,"物販, ファッション・雑貨, コンビニ・ドラッグストア, スーパーマーケット, アミューズメ..."
87,第１富澤ビル５Ｆ,東京都新宿区四谷３丁目１－３,東京メトロ丸ノ内線 四谷三丁目,121.8646,281.71,
88,第３富澤ビル２Ｆ,東京都新宿区四谷３丁目２－７,東京メトロ丸ノ内線 四谷三丁目,27.038,81.25,


PermissionError: [Errno 13] Permission denied: '東京都新宿区の貸店舗・テナント物件一覧_店舗ネットワーク.csv'